In [1]:
import numpy as np
import pandas as pd

from utils import evaluate_gap,collate_batch_ptr,split_train_test
from local_search import local_search_predict
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.nn as nn
import torch
import time

from multiprocessing import get_context


%reload_ext autoreload
%autoreload 2


/opt/conda/envs/instadeep/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else  "cpu"
print(DEVICE)

cuda


# Evaluation Metric

**Gap**: $100*\frac{model\_time - optimal\_time}{optimal\_time}$

# Load Data

In [4]:
instances = np.load('data/instances.npy')
instances_orders = np.load('data/instances_orders_unpadded.npy')

X,y = instances,instances_orders

# Unpad data

In [9]:
from utils import unpad

for seq in y:
    seq = unpad(seq)

In [38]:
np.save('data/instances_orders_unpadded.npy',y)

# Split data

In [5]:
train_dataset,test_dataset = split_train_test(X,y,0.1)

# Solving TSP with Local Search

In [11]:
test_dataloader = DataLoader(test_dataset,batch_size=256)

In [16]:
y_preds = []
total_gaps = 0
total = 0

for x,y in tqdm(test_dataloader,total = len(test_dataloader)):
    total += len(y)

    with get_context("spawn").Pool() as pool:
                        preds = pool.map(
                            local_search_predict,x)
                        gaps = pool.starmap(
                                evaluate_gap, list(zip(preds,y,x))
                                
                        )
    y_preds.append(preds)
    total_gaps+= np.sum(gaps)
    
print(f"AVERAGE GAP FOR LOCAL SEARCH SOLVER: {total_gaps/total}")




100%|██████████| 43/43 [03:47<00:00,  5.29s/it]

AVERAGE GAP FOR LOCAL SEARCH SOLVER: 10.819183554028097


# PointerNets

In [6]:
import torch
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from torch.utils.data import DataLoader

import numpy as np
import argparse
from tqdm import tqdm

from pointernet import PointerNet
from Data_Generator import TSPDataset

In [26]:
LR = 10.0
BATCH_SIZE = 16
NUM_EPOCHS = 5

In [27]:
model = PointerNet(128,
                   512,
                   2,
                   0.1,
                   False).to(DEVICE)

train_dataloader = DataLoader(train_dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        collate_fn=collate_batch_ptr)

test_dataloader = DataLoader(test_dataset,
                        batch_size=BATCH_SIZE,
                        shuffle=True,
                        collate_fn=collate_batch_ptr)

criterion = torch.nn.CrossEntropyLoss().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(),lr = LR)

In [28]:
def train_ptr(dataloader, model, optimizer, criterion, epoch):
    model.train()
    total_loss,total_gap, total_count = 0,0,0
    log_interval = 100

    for idx, (x, y) in tqdm(enumerate(dataloader)):
        optimizer.zero_grad()
        total_count += y.shape[0]

        y_pred,pointers = model(x)
        y_pred =  y_pred.contiguous().view(-1, y_pred.size()[-1])

        # Get the loss
        loss = criterion(y_pred,y.view(-1))
        total_loss  += loss.item()
        # Do back propagation
        loss.backward()
        # Clip the gradients at 0.1
        nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        # Do an optimization step
        optimizer.step()
        for i,pred in enumerate(pointers.cpu().detach().numpy()):
            total_gap += evaluate_gap(pred,y[i].cpu().detach().numpy(),x[i].cpu())
        if idx % log_interval == 0 and idx > 0:
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches"
                "| gap {:8.3f}".format(epoch, idx, len(dataloader), total_gap / total_count),
                f"| loss {total_loss/total_count}"
            )
            total_gap, total_count,total_loss = 0, 0, 0

In [29]:
for epoch in range(1, NUM_EPOCHS + 1):
    epoch_start_time = time.time()
    train_ptr(train_dataloader, model, optimizer, criterion, epoch)
    #accu_val = evaluate(valid_dataloader, model)
    #scheduler.step()
    print("-" * 59)
    #print(
    #    "| end of epoch {:3d} | time: {:5.2f}s | "
    #    "valid accuracy {:8.3f} ".format(epoch, time.time() - epoch_start_time, accu_val)
    #)
    #print("-" * 59)

#print("Checking the results of test dataset.")
#accu_test = evaluate(test_dataloader, model)
#print("test accuracy {:8.3f}".format(accu_test))

0it [00:00, ?it/s]

102it [00:14,  7.51it/s]

| epoch   1 |   100/ 6104 batches| gap  325.773 | loss 0.24334930311335196


202it [00:28,  7.05it/s]

| epoch   1 |   200/ 6104 batches| gap  235.157 | loss 0.24059342756867408


...

| epoch   1 |  2700/ 6104 batches| gap  175.711 | loss 0.24008295834064483


2802it [06:23,  7.51it/s]

| epoch   1 |  2800/ 6104 batches| gap  178.059 | loss 0.2401403945684433


2848it [06:29,  7.30it/s]


KeyboardInterrupt: 

# Transformer Net

In [86]:
from transformernet import TSP_net

In [89]:
test_loader = DataLoader(test_dataset,
                        batch_size=16,
                        shuffle=True,
                        collate_fn=collate_batch_ptr)

In [90]:
model_train = TSP_net(2, 128, 512, 
              6, 2, 8, 1000,
              False)

In [92]:
for x, y in test_loader:
    print(x.shape)
    y_pred = model_train(x)
    print(y_pred[0][1])
    print(y[1])
    print(x[1])
    break

torch.Size([16, 50, 2])
tensor([37, 30,  3, 28,  5, 26, 27,  1, 24, 29, 19, 21, 31,  9, 20, 22, 35, 23,
         4, 14, 11, 34,  7, 16, 33,  2, 17,  6, 12,  8, 32, 36, 10, 15, 18, 45,
        40, 25, 13,  0, 48, 43, 42, 46, 39, 47, 38, 44, 41, 49])
tensor([ 0, 10, 15, 31, 27,  3, 13, 18, 22, 29, 24, 12, 17, 14,  2, 33, 23, 36,
        20,  4, 16,  1, 11, 26, 30, 28,  9, 32, 21,  6, 34, 35,  7,  8, 25, 19,
         5, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])
tensor([[ 0.0043,  0.0529],
        [ 0.5244,  0.9983],
        [ 0.2644,  0.6819],
        [ 0.1576,  0.4723],
        [ 0.3675,  0.9451],
        [ 0.1551,  0.1053],
        [ 0.7382,  0.5765],
        [ 0.7900,  0.0018],
        [ 0.7367,  0.0015],
        [ 0.9485,  0.9472],
        [ 0.0513,  0.3122],
        [ 0.5290,  0.7354],
        [ 0.5295,  0.3148],
        [ 0.2128,  0.4243],
        [ 0.3147,  0.4225],
        [ 0.1025,  0.3678],
        [ 0.4738,  1.0000],
        [ 0.5299,  0.4747],
        [ 0.2073,  0.2